# 07: 完全なTransformerモデル

このノートブックでは、EncoderとDecoderを統合した**完全なTransformerモデル**を学習します。

## 目次
1. Transformerの全体構造
2. モデルの作成と順伝播
3. シーケンス生成（推論）
4. パラメータ数の分析
5. 簡単な学習タスク（コピータスク）

In [ ]:
import sys
sys.path.append('..')

import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

# 日本語フォントの設定（macOS）
import matplotlib
matplotlib.rcParams['font.family'] = 'Hiragino Sans'
matplotlib.rcParams['axes.unicode_minus'] = False  # マイナス記号の文字化け対策

import numpy as np

from src.transformer import Transformer, count_parameters

# デバイスの設定
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')
print(f"Using device: {device}")

## 1. Transformerの全体構造

```
              ソース入力              ターゲット入力
             (日本語文)               (英語文)
                 ↓                       ↓
         ┌─────────────┐          ┌─────────────┐
         │ Embedding   │          │ Embedding   │
         └──────┬──────┘          └──────┬──────┘
                ↓                        ↓
         ┌─────────────┐          ┌─────────────┐
         │ + Position  │          │ + Position  │
         │  Encoding   │          │  Encoding   │
         └──────┬──────┘          └──────┬──────┘
                ↓                        ↓
        ┌───────────────┐        ┌───────────────┐
        │               │        │ Masked Self- │
        │  Self-Attn    │        │  Attention   │
        │               │        │              │
        ├───────────────┤        ├──────────────┤
        │  Add & Norm   │        │ Add & Norm   │
        ├───────────────┤        ├──────────────┤
        │               │   K,V  │              │
        │     FFN       │───────→│ Cross-Attn   │
        │               │        │              │
        ├───────────────┤        ├──────────────┤
        │  Add & Norm   │        │ Add & Norm   │
        └───────┬───────┘        ├──────────────┤
                ↓                │     FFN      │
                ↓                ├──────────────┤
           × N layers            │ Add & Norm   │
                                 └──────┬───────┘
                                        ↓
                                   × N layers
                                        ↓
                                 ┌─────────────┐
                                 │   Linear    │
                                 │  (vocab)    │
                                 └──────┬──────┘
                                        ↓
                                 ┌─────────────┐
                                 │   Softmax   │
                                 └──────┬──────┘
                                        ↓
                                    出力確率
```

## 2. モデルの作成と順伝播

In [ ]:
# パラメータ設定
src_vocab_size = 1000  # ソース語彙サイズ
tgt_vocab_size = 1000  # ターゲット語彙サイズ
d_model = 256          # モデル次元
num_heads = 8          # Attentionヘッド数
num_layers = 4         # Encoder/Decoder層数
d_ff = 1024            # FFN中間層次元

# モデルの作成
model = Transformer(
    src_vocab_size=src_vocab_size,
    tgt_vocab_size=tgt_vocab_size,
    d_model=d_model,
    num_heads=num_heads,
    num_encoder_layers=num_layers,
    num_decoder_layers=num_layers,
    d_ff=d_ff,
).to(device)

print(f"Total parameters: {count_parameters(model):,}")

In [ ]:
# ダミー入力で順伝播
batch_size = 2
src_len = 10
tgt_len = 8

# ランダムなトークンID（0はパディング、1以上が実際のトークン）
src = torch.randint(1, src_vocab_size, (batch_size, src_len)).to(device)
tgt = torch.randint(1, tgt_vocab_size, (batch_size, tgt_len)).to(device)

print(f"Source shape: {src.shape}")
print(f"Target shape: {tgt.shape}")

# 順伝播
model.train()
logits = model(src, tgt)

print(f"\nOutput logits shape: {logits.shape}")
print(f"  - batch_size: {logits.shape[0]}")
print(f"  - tgt_len: {logits.shape[1]}")
print(f"  - vocab_size: {logits.shape[2]}")

In [ ]:
# 確率分布に変換
probs = F.softmax(logits, dim=-1)

# 最初のバッチ、最初の位置の確率分布を可視化
plt.figure(figsize=(12, 4))
plt.bar(range(50), probs[0, 0, :50].detach().cpu().numpy())
plt.xlabel('Token ID')
plt.ylabel('Probability')
plt.title('Output probability distribution (first 50 tokens)')
plt.tight_layout()
plt.show()

print(f"Most likely token: {probs[0, 0].argmax().item()}")
print(f"Top-5 tokens: {probs[0, 0].topk(5).indices.tolist()}")

## 3. シーケンス生成（推論）

学習済みモデルを使って、ソースから新しいシーケンスを生成します。

### 生成方法
1. **Greedy（貪欲法）**: 各位置で最も確率の高いトークンを選択
2. **サンプリング**: 確率分布からランダムにサンプリング
3. **Top-K**: 上位K個のトークンからサンプリング
4. **Top-P (Nucleus)**: 累積確率がP以下のトークンからサンプリング

In [ ]:
# Greedy生成
model.eval()
with torch.no_grad():
    generated_greedy = model.greedy_decode(
        src, 
        max_len=15,
        start_token_id=1,  # <start>
        end_token_id=2,    # <end>
    )

print("Greedy generation:")
print(f"  Shape: {generated_greedy.shape}")
print(f"  Batch 0: {generated_greedy[0].tolist()}")
print(f"  Batch 1: {generated_greedy[1].tolist()}")

In [ ]:
# サンプリング生成（温度付き）
with torch.no_grad():
    # 温度 0.5（より確定的）
    generated_temp05 = model.generate(
        src,
        max_len=15,
        temperature=0.5,
    )
    
    # 温度 1.5（よりランダム）
    generated_temp15 = model.generate(
        src,
        max_len=15,
        temperature=1.5,
    )

print("Temperature 0.5 (more deterministic):")
print(f"  Batch 0: {generated_temp05[0].tolist()}")

print("\nTemperature 1.5 (more random):")
print(f"  Batch 0: {generated_temp15[0].tolist()}")

In [ ]:
# Top-K サンプリング
with torch.no_grad():
    generated_topk = model.generate(
        src,
        max_len=15,
        top_k=10,
    )

print("Top-K (K=10) sampling:")
print(f"  Batch 0: {generated_topk[0].tolist()}")

## 4. パラメータ数の分析

In [ ]:
# 各コンポーネントのパラメータ数を計算
def get_param_breakdown(model):
    breakdown = {
        'Source Embedding': 0,
        'Target Embedding': 0,
        'Position Encoding': 0,
        'Encoder': 0,
        'Decoder': 0,
        'Output Projection': 0,
    }
    
    for name, param in model.named_parameters():
        if 'src_embedding' in name:
            breakdown['Source Embedding'] += param.numel()
        elif 'tgt_embedding' in name:
            breakdown['Target Embedding'] += param.numel()
        elif 'pos_encoding' in name:
            breakdown['Position Encoding'] += param.numel()
        elif 'encoder' in name:
            breakdown['Encoder'] += param.numel()
        elif 'decoder' in name:
            breakdown['Decoder'] += param.numel()
        elif 'output_projection' in name:
            breakdown['Output Projection'] += param.numel()
    
    return breakdown

breakdown = get_param_breakdown(model)
total = sum(breakdown.values())

print("Parameter Breakdown:")
print("=" * 50)
for name, count in breakdown.items():
    pct = 100 * count / total if total > 0 else 0
    print(f"{name:20s}: {count:>10,} ({pct:5.1f}%)")
print("=" * 50)
print(f"{'Total':20s}: {total:>10,}")

In [ ]:
# パイチャートで可視化
fig, ax = plt.subplots(figsize=(10, 8))

# 0のエントリを除外
labels = [k for k, v in breakdown.items() if v > 0]
sizes = [v for v in breakdown.values() if v > 0]

colors = plt.cm.Set3(np.linspace(0, 1, len(labels)))
explode = [0.02] * len(labels)

wedges, texts, autotexts = ax.pie(
    sizes, 
    labels=labels, 
    autopct='%1.1f%%',
    colors=colors,
    explode=explode,
    startangle=90,
)

ax.set_title(f'Transformer Parameter Distribution\nTotal: {total:,} parameters')
plt.tight_layout()
plt.show()

In [ ]:
# 論文オリジナル設定との比較
print("Comparison with different model sizes:")
print("=" * 60)

configs = [
    ("Current (small)", 256, 4, 1024, 1000),
    ("Transformer-Base", 512, 6, 2048, 32000),
    ("Transformer-Big", 1024, 6, 4096, 32000),
]

for name, d, layers, ff, vocab in configs:
    temp_model = Transformer(
        src_vocab_size=vocab,
        tgt_vocab_size=vocab,
        d_model=d,
        num_heads=8,
        num_encoder_layers=layers,
        num_decoder_layers=layers,
        d_ff=ff,
    )
    params = count_parameters(temp_model)
    print(f"{name:20s}: d={d:4d}, layers={layers}, vocab={vocab:5d} -> {params:>12,} params")
    del temp_model

## 5. 簡単な学習タスク（コピータスク）

入力シーケンスをそのまま出力するタスクでTransformerを学習させます。

```
入力:  [3, 5, 7, 2, 9]
出力:  [3, 5, 7, 2, 9]
```

In [ ]:
# コピータスク用のデータ生成
def generate_copy_data(batch_size, seq_len, vocab_size, pad_idx=0, start_idx=1, end_idx=2):
    """
    コピータスクのデータを生成
    
    ソース: [random tokens]
    ターゲット入力: [<start>, random tokens]
    ターゲット出力: [random tokens, <end>]
    """
    # ランダムなトークン（3以降を使用、0=pad, 1=start, 2=end）
    tokens = torch.randint(3, vocab_size, (batch_size, seq_len))
    
    # ソース
    src = tokens.clone()
    
    # ターゲット入力: <start> + tokens
    tgt_input = torch.cat([
        torch.full((batch_size, 1), start_idx),
        tokens
    ], dim=1)
    
    # ターゲット出力（教師信号）: tokens + <end>
    tgt_output = torch.cat([
        tokens,
        torch.full((batch_size, 1), end_idx)
    ], dim=1)
    
    return src, tgt_input, tgt_output

# テスト
src, tgt_in, tgt_out = generate_copy_data(2, 5, 20)
print("Source:       ", src[0].tolist())
print("Target Input: ", tgt_in[0].tolist())
print("Target Output:", tgt_out[0].tolist())

In [ ]:
# コピータスク用の小さなモデル
copy_vocab_size = 20
copy_model = Transformer(
    src_vocab_size=copy_vocab_size,
    tgt_vocab_size=copy_vocab_size,
    d_model=64,
    num_heads=4,
    num_encoder_layers=2,
    num_decoder_layers=2,
    d_ff=256,
    src_pad_idx=0,
    tgt_pad_idx=0,
).to(device)

print(f"Copy model parameters: {count_parameters(copy_model):,}")

In [ ]:
# 学習設定
optimizer = torch.optim.Adam(copy_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=0)  # パディングは無視

# 学習ループ
num_epochs = 100
batch_size = 32
seq_len = 8

losses = []
accuracies = []

copy_model.train()
for epoch in range(num_epochs):
    # データ生成
    src, tgt_in, tgt_out = generate_copy_data(batch_size, seq_len, copy_vocab_size)
    src = src.to(device)
    tgt_in = tgt_in.to(device)
    tgt_out = tgt_out.to(device)
    
    # 順伝播
    logits = copy_model(src, tgt_in)
    
    # 損失計算
    loss = criterion(
        logits.reshape(-1, copy_vocab_size),
        tgt_out.reshape(-1)
    )
    
    # 精度計算
    predictions = logits.argmax(dim=-1)
    correct = (predictions == tgt_out).float().mean().item()
    
    # 逆伝播
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())
    accuracies.append(correct)
    
    if (epoch + 1) % 20 == 0:
        print(f"Epoch {epoch+1:3d}: Loss = {loss.item():.4f}, Accuracy = {correct:.4f}")

In [ ]:
# 学習曲線の可視化
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(losses)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training Loss')
ax1.grid(True, alpha=0.3)

ax2.plot(accuracies)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.set_title('Training Accuracy')
ax2.set_ylim(0, 1.05)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 学習後のテスト
copy_model.eval()

# テストデータ
test_src, test_tgt_in, test_tgt_out = generate_copy_data(5, seq_len, copy_vocab_size)
test_src = test_src.to(device)

print("Copy Task Test Results:")
print("=" * 50)

with torch.no_grad():
    generated = copy_model.greedy_decode(
        test_src,
        max_len=seq_len + 2,
        start_token_id=1,
        end_token_id=2,
    )

for i in range(5):
    src_tokens = test_src[i].tolist()
    gen_tokens = generated[i].tolist()
    # <start>と<end>を除去して比較
    gen_clean = [t for t in gen_tokens[1:] if t != 2][:seq_len]
    
    match = "✓" if src_tokens == gen_clean else "✗"
    print(f"{match} Input:  {src_tokens}")
    print(f"  Output: {gen_clean}")
    print()

## まとめ

### 学習したこと

1. **Transformerの構造**
   - Encoder: Self-Attention → FFN を N層
   - Decoder: Masked Self-Attention → Cross-Attention → FFN を N層
   - Cross-Attentionでソースとターゲットを接続

2. **順伝播の流れ**
   - ソース → Encoder → Encoder出力
   - ターゲット + Encoder出力 → Decoder → 語彙上の確率分布

3. **シーケンス生成**
   - Greedy: 常に最大確率のトークンを選択
   - サンプリング: 確率分布からランダムに選択
   - Temperature: 確率分布の鋭さを調整
   - Top-K/Top-P: 候補を絞ってからサンプリング

4. **パラメータ配分**
   - Embedding層: 語彙サイズ × d_model
   - Encoder/Decoder: 層数に比例
   - Decoderの方がCross-Attention分多い

### 次のステップ
- 実際の翻訳タスクでの学習
- Beam Search の実装
- 事前学習モデル（BERT, GPT）との比較